In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.ensemble import IsolationForest

In [3]:
# sample_path = "../../../logs/cyberattack_status_0_0.005_c_0_0.csv" # cyberattack_status_0_0.005_c_0_0.csv

# df = pd.read_csv(sample_path)

# read a sample:
# sample_path = "../code/logs/charging_od_0_0.005_c_0.csv"

# cyberattack_status_0_0.005_c_0_0.csv

df = pd.read_csv("../../../logs/parsed_results_0_0.005_c_19_1_0.csv")
df_attack = pd.read_csv("../../../logs/parsed_results_0_0.005_c_19_1_5.csv")

In [4]:
df["hour"] = df["time"]/60%57600%1440%24
df["hour"]

0        0.0
1        1.0
2        2.0
3        3.0
4        4.0
        ... 
1435    19.0
1436    20.0
1437    21.0
1438    22.0
1439    23.0
Name: hour, Length: 1440, dtype: float64

In [5]:
df[" unit_system_revenue"]  = df[" total_system_revenue"].diff()
df = df[df["hour"].isin([6,7,8,9,17,18,19,20])]

In [6]:
# parse a sample of hourly statistics
normal_profile = df[["hour"," num_idle"," num_serving"," num_charging"," num_assigned"," num_waitpile"," unit_system_revenue"]].rolling(30).mean().dropna()
# normal_profile = normal_profile.iloc[120:,]
normal_profile = normal_profile[normal_profile[" unit_system_revenue"]>0]

In [7]:
random_state = np.random.RandomState(1)

model=IsolationForest(n_estimators=100,max_samples='auto',contamination=float(0.05),random_state=random_state) # a false alarm rate of 5%.

model.fit(normal_profile)

c:\Users\sguo18\Anaconda3\envs\dp\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


IsolationForest(contamination=0.05,
                random_state=RandomState(MT19937) at 0x1A95DFFB840)

In [8]:
normal_profile['scores'] = model.decision_function(normal_profile)

normal_profile['is_inlier'] = model.predict(normal_profile.iloc[:,:-1])

In [11]:
normal_profile[normal_profile['is_inlier']==-1].shape

(23, 9)

In [13]:
df_attack[" unit_system_revenue"] = df_attack[" total_system_revenue"].diff()
df_attack["hour"] = df_attack["time"]/60%57600%1440%24
df_attack = df_attack[df_attack["hour"].isin([6,7,8,9,17,18,19,20])]

attacked_profile = df_attack[["hour"," num_idle"," num_serving"," num_charging"," num_assigned"," num_waitpile"," unit_system_revenue"]].rolling(30).mean().dropna()
attacked_profile = attacked_profile[attacked_profile[" unit_system_revenue"]>0]

In [14]:
attacked_profile['scores'] = model.decision_function(attacked_profile)

attacked_profile['is_inlier'] = model.predict(attacked_profile.iloc[:,:-1])

attacked_profile[attacked_profile['is_inlier']==-1]


,hour,num_idle,num_serving,num_charging,num_assigned,num_waitpile,unit_system_revenue,scores,is_inlier
452,13.433333,93.533333,431.900000,81.333333,277.266667,3.633333,0.249244,-0.002116,-1
473,12.633333,105.600000,419.100000,81.566667,247.366667,3.733333,0.257841,-0.000313,-1
474,12.566667,108.800000,416.533333,81.400000,239.600000,3.666667,0.251172,-0.009132,-1
475,13.000000,111.533333,412.766667,81.466667,232.300000,3.833333,0.232354,-0.003571,-1
476,13.433333,114.966667,408.600000,81.666667,225.100000,3.900000,0.237093,-0.028592,-1
486,13.366667,118.266667,403.666667,81.633333,218.400000,3.933333,0.248918,-0.030648,-1
487,13.300000,121.533333,398.366667,81.666667,211.833333,3.833333,0.259876,-0.046627,-1
488,13.000000,124.700000,392.500000,81.600000,206.733333,3.700000,0.267337,-0.033099,-1
489,12.700000,127.433333,386.566667,81.400000,202.466667,3.700000,0.273625,-0.039937,-1
497,12.633333,128.933333,379.400000,81.366667,201.666667,3.666667,0.274207,-0.052658,-1


In [15]:
attacked_profile[attacked_profile['is_inlier']==-1].shape

(33, 9)